##### CityLearnART built using these commands
- conda create -n CityLearnART python=3.10 setuptools==66 wheel<0.40 numpy pytorch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0 cudatoolkit=11.6 ipywidgets -c pytorch -c conda-forge
- conda activate CityLearnART
- pip install adversarial-robustness-toolbox[pytorch] citylearn==2.0b5 stable-baselines3[extra]==1.8.0

restrictions (painful lessons):
- CityLearn only works with PyTorch v1
- CityLearn only works with SB3<2 ref: https://github.com/intelligent-environments-lab/CityLearn/issues/63, for this reason https://gymnasium.farama.org/content/migration-guide/#environment-reset
- SB3 <2 requires gym 0.21 
- gym requires setuptools==66 and wheel<0.40 ref:https://github.com/openai/gym/issues/3200, https://github.com/openai/gym/issues/3211, https://stackoverflow.com/questions/76129688/why-is-pip-install-gym-failing-with-python-setup-py-egg-info-did-not-run-succ 
- ART requires a version of numpy later than 1.21

In [1]:
from stable_baselines3 import A2C #Approaches SAC's performance in 2021 challenge benchmark, though PPO did well in this example: https://www.aicrowd.com/showcase/going-below-1-0-score-with-stablebaseline3

from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper, DiscreteActionWrapper
from citylearn.data import DataSet

from typing import List
import numpy as np

from art.utils import to_categorical
from art.attacks.evasion.brendel_bethge import BrendelBethgeAttack

from KBMproject.mybb import BrendelBethgeAttack as mybba
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
#dataset_name = 'baeda_3dem' #only in CityLearn v2
dataset_name = 'citylearn_challenge_2022_phase_1' #only action is electrical storage

In [3]:
def print_active_actions(schema: dict):
    """prints active actions in a CityLearn schema"""
    print('Active actions:')
    for key, value in schema['actions'].items():
        if value['active']:
            print('\t',key)

In [4]:
schema = DataSet.get_schema(dataset_name)
print_active_actions(schema)

Active actions:
	 electrical_storage


Will ART's regressor work with a 1d action space? Ref for code below: https://colab.research.google.com/drive/1rZn6qLEIHMlu2iwNl1jKqvcEet8lS33A#scrollTo=A04LckaH0uuS

In [5]:
def set_active_actions(
    schema: dict, active_actions: List[str]
) -> dict:
    """Set the actions that will be part of the environment's 
    observation space that is provided to the control agent.

    Parameters
    ----------
    schema: dict
        CityLearn dataset mapping used to construct environment.
    active_actions: List[str]
        Names of actions to set active to be passed to control agent.

    Returns
    -------
    schema: dict
        CityLearn dataset mapping with active actions set.
    """

    active_count = 0

    for o in schema['actions']:
        if o in active_actions:
            schema['actions'][o]['active'] = True
            active_count += 1
        else:
            schema['actions'][o]['active'] = False

    valid_actions = list(schema['actions'].keys())
    assert active_count == len(active_actions),\
        'the provided actions are not all valid actions.'\
          f' Valid actionss in CityLearn are: {valid_actions}'
    
    return schema

In [6]:
#This demonstrates how to change the active action, but this is unnecessary for citylearn_challenge_2022_phase_1
#schema = set_active_actions(schema, ['electrical_storage']) 
#print_active_actions(schema)

By removing active actions we ensure that there is only one device to control in each building, however multiple buildings will still result in the agent controlling multiple devices, so we mut remove all but one building.

In [7]:
list(schema['buildings'].keys())

['Building_1', 'Building_2', 'Building_3', 'Building_4', 'Building_5']

In [8]:
def del_buildings(
schema: dict, buildings_to_del: List[str]
) -> dict:
        """removes buildings from a CityLeanr Schema.

    Parameters
    ----------
    schema: dict
        CityLearn dataset mapping used to construct environment.
    buildings_to_del: List[str]
        Names of buildings to be removed from the environment.

    Returns
    -------
    schema: dict
        CityLearn dataset mapping with buildings removed.
    """

        print('Buildings in original schema: ', list(schema['buildings'].keys()))
        schema['buildings'] = {key: value for key, value in schema['buildings'].items() if key not in buildings_to_del}
        print('Buildings in new schema: ', list(schema['buildings'].keys()))

        return schema

Remove all but building 1

In [9]:
schema = del_buildings(schema, list(schema['buildings'].keys())[1:])

Buildings in original schema:  ['Building_1', 'Building_2', 'Building_3', 'Building_4', 'Building_5']
Buildings in new schema:  ['Building_1']


In [10]:
#set env parameters
env = CityLearnEnv(schema, 
                   central_agent=True, #necessary for single agent
                   #simulation_end_time_step=1000
                   )
#wrap env
env = DiscreteActionWrapper(env, 
                            bin_sizes=[{'electrical_storage':20}] #this must be a list dicts for each action, see TabularQLearningWrapper example from https://colab.research.google.com/drive/1rZn6qLEIHMlu2iwNl1jKqvcEet8lS33A#scrollTo=1rkt9jnNuiZE
                            )
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
#Set RL algo parameters
policy_kwargs = dict(net_arch=[256, 256])
agent = A2C('MlpPolicy', 
            env,
            device='cuda',
            policy_kwargs=policy_kwargs)

In [11]:
help(CityLearnEnv)

Help on class CityLearnEnv in module citylearn.citylearn:

class CityLearnEnv(citylearn.base.Environment, gym.core.Env)
 |  CityLearnEnv(schema: Union[str, pathlib.Path, Mapping[str, Any]], root_directory: Union[str, pathlib.Path] = None, buildings: Union[List[citylearn.building.Building], List[str], List[int]] = None, simulation_start_time_step: int = None, simulation_end_time_step: int = None, episode_time_steps: Union[int, List[Tuple[int, int]]] = None, rolling_episode_split: bool = None, random_episode_split: bool = None, seconds_per_time_step: float = None, reward_function: citylearn.reward_function.RewardFunction = None, central_agent: bool = None, shared_observations: List[str] = None, random_seed: int = None, **kwargs: Any)
 |  
 |  CityLearn nvironment class.
 |  
 |  Parameters
 |  ----------
 |  schema: Union[str, Path, Mapping[str, Any]]
 |      Name of CityLearn data set, filepath to JSON representation or :code:`dict` object of a CityLearn schema.
 |      Call :py:meth:`c

In [12]:
env.time_steps

8760

Display the default reward function

In [13]:
help(env.reward_function)

Help on RewardFunction in module citylearn.reward_function object:

class RewardFunction(builtins.object)
 |  RewardFunction(env_metadata: Mapping[str, Any], **kwargs)
 |  
 |  Base and default reward function class.
 |  
 |  The default reward is the electricity consumption from the grid at the current time step returned as a negative value.
 |  
 |  Parameters
 |  ----------
 |  env_metadata: Mapping[str, Any]:
 |      General static information about the environment.
 |  **kwargs : dict
 |      Other keyword arguments for custom reward calculation.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, env_metadata: Mapping[str, Any], **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  calculate(self, observations: List[Mapping[str, Union[int, float]]]) -> List[float]
 |      Calculates reward.
 |      
 |      Parameters
 |      ----------
 |      observations: List[Mapping[str, Union[int, float]]]
 |          List of all building observations 

Confirm the action space is 1d for compatibility with ART:

In [14]:
assert env.action_space.shape[0] == 1, f'action space is {env.action_space.shape[0]} dimensional, but should be 1' 

In [15]:
agent.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=31, out_features=256, bias=True)
      (1): Tanh()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=31, out_features=256, bias=True)
      (1): Tanh()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=256, out_features=20, bias=True)
  (value_net): Linear(in_features=256, out_features=1, bias=True)
)

In [16]:
agent.policy.mlp_extractor.policy_net

Sequential(
  (0): Linear(in_features=31, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Tanh()
)

In [17]:
agent.policy.action_net

Linear(in_features=256, out_features=20, bias=True)

In [18]:
from copy import deepcopy
policy_net = deepcopy(agent.policy.mlp_extractor.policy_net) #copies shared net rather than referencing
policy_net.add_module('4', agent.policy.action_net)
policy_net

Sequential(
  (0): Linear(in_features=31, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Tanh()
  (4): Linear(in_features=256, out_features=20, bias=True)
)

Initialization specifically for the PyTorch-based implementation:
https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/estimators/classification.html#pytorch-classifier

In [19]:
from art.estimators.classification import PyTorchClassifier as classifier
from torch.nn import CrossEntropyLoss

victim_policy = classifier(
    model=policy_net,
    loss=CrossEntropyLoss(), #most common func for classification
    nb_classes=env.action_space[0].n,
    input_shape=agent.observation_space.shape,
    device_type='gpu',
    clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max())#(agent.observation_space.low,agent.observation_space.high) # arrays of  min and max values of each feature break the brendle bethge attack
    )

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [20]:
obs = env.reset()

In [21]:
help(classifier)

Help on class PyTorchClassifier in module art.estimators.classification.pytorch:

class PyTorchClassifier(art.estimators.classification.classifier.ClassGradientsMixin, art.estimators.classification.classifier.ClassifierMixin, art.estimators.pytorch.PyTorchEstimator)
 |  PyTorchClassifier(model: 'torch.nn.Module', loss: 'torch.nn.modules.loss._Loss', input_shape: Tuple[int, ...], nb_classes: int, optimizer: Optional[ForwardRef('torch.optim.Optimizer')] = None, use_amp: bool = False, opt_level: str = 'O1', loss_scale: Union[float, str, NoneType] = 'dynamic', channels_first: bool = True, clip_values: Optional[ForwardRef('CLIP_VALUES_TYPE')] = None, preprocessing_defences: Union[ForwardRef('Preprocessor'), List[ForwardRef('Preprocessor')], NoneType] = None, postprocessing_defences: Union[ForwardRef('Postprocessor'), List[ForwardRef('Postprocessor')], NoneType] = None, preprocessing: 'PREPROCESSING_TYPE' = (0.0, 1.0), device_type: str = 'gpu') -> None
 |  
 |  This class implements a classi

In [22]:
from art.attacks.evasion import AutoProjectedGradientDescent as APGD
from art.attacks.evasion import AutoConjugateGradient as ACG

APGDatk = APGD(estimator=victim_policy, verbose=False)
ACGatk = ACG(estimator=victim_policy, verbose=False)

In [23]:
agent.observation_space.shape == obs.shape

True

generate attack using initial observation to verify pipeline

Same issue as I had in sinergym, need a different branch to solve the issue ref: https://github.com/Trusted-AI/adversarial-robustness-toolbox/issues/2165

The input must have shape (n_sample, n_features), as obs is a 1d array of features with shape (20,) expand dims adds the n_samples for a shape of (1,20)

In [24]:
APGDatk.generate(np.expand_dims(obs, axis=0))

array([[0.36698732, 0.        , 0.5535534 , 0.        , 1.        ,
        0.19999999, 0.2413534 , 0.77744365, 0.3466165 , 0.2413534 ,
        0.5222222 , 1.        , 0.34444445, 0.48888886, 0.        ,
        0.        , 1.        , 0.3       , 0.3       , 0.        ,
        0.55519414, 0.3       , 0.17462254, 0.        , 0.        ,
        0.        , 0.6901262 , 0.        , 0.        , 0.33030304,
        0.33030304]], dtype=float32)

In [25]:
adv_obs = ACGatk.generate(np.expand_dims(obs, axis=0))

In [26]:
adv_obs

array([[0.30530447, 0.24474017, 0.63920146, 0.12466008, 1.        ,
        0.78491527, 0.40792447, 0.77744365, 0.3466165 , 0.36696437,
        0.7207528 , 1.        , 0.5241754 , 0.9510994 , 0.22348298,
        0.32458213, 1.        , 0.11122806, 0.3       , 0.35101303,
        0.9915317 , 0.14146215, 0.77462256, 0.43223327, 0.09068313,
        0.08863807, 0.6901262 , 0.31768602, 0.        , 0.33030304,
        0.02581663]], dtype=float32)

In [27]:
agent.predict(np.squeeze(adv_obs), deterministic=True)

(array([5], dtype=int64), None)

In [28]:
np.argmax(victim_policy.predict(adv_obs, training_mode=False))

5

In [29]:
agent.action_space

MultiDiscrete([20])

In [30]:
agent.action_space[0].n

20

In [31]:
ACGtarg = ACG(estimator=victim_policy, targeted=True, verbose=False)

In [32]:
from art.utils import to_categorical
targ_obs = ACGtarg.generate(np.expand_dims(obs, axis=0), to_categorical([19], nb_classes=agent.action_space[0].n) )
agent.predict(np.squeeze(targ_obs), deterministic=True)

(array([9], dtype=int64), None)

In [33]:
agent.predict(np.squeeze(obs), deterministic=True)

(array([9], dtype=int64), None)

In [34]:
help(BrendelBethgeAttack)
bba = BrendelBethgeAttack(estimator=victim_policy)

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


Help on class BrendelBethgeAttack in module art.attacks.evasion.brendel_bethge:

class BrendelBethgeAttack(art.attacks.attack.EvasionAttack)
 |  BrendelBethgeAttack(estimator: 'CLASSIFIER_LOSS_GRADIENTS_TYPE', norm: Union[int, float, str] = inf, targeted: bool = False, overshoot: float = 1.1, steps: int = 1000, lr: float = 0.001, lr_decay: float = 0.5, lr_num_decay: int = 20, momentum: float = 0.8, binary_search_steps: int = 10, init_size: int = 100, batch_size: int = 32)
 |  
 |  Base class for the Brendel & Bethge adversarial attack [#Bren19]_, a powerful gradient-based adversarial attack that
 |  follows the adversarial boundary (the boundary between the space of adversarial and non-adversarial images as
 |  defined by the adversarial criterion) to find the minimum distance to the clean image.
 |  
 |  This is implementation of the Brendel & Bethge attack follows the reference implementation at
 |  https://github.com/bethgelab/foolbox/blob/master/foolbox/attacks/brendel_bethge.py.
 

In [35]:
bba.generate(np.expand_dims(obs, axis=0))


INFO:art.attacks.evasion.brendel_bethge:Using model predictions as correct labels for FGM.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for untargeted attack.


array([[0.        , 0.29496175, 0.71825564, 0.01114884, 1.        ,
        0.6352978 , 0.6766512 , 0.5470734 , 0.7819143 , 0.40605563,
        0.68692446, 0.6535911 , 0.5091467 , 0.6535911 , 0.0772858 ,
        0.15987988, 1.        , 0.13529778, 0.13529778, 0.        ,
        0.78132814, 0.13529778, 0.6099203 , 0.14448342, 0.        ,
        0.13529778, 0.8548284 , 0.1656008 , 0.        , 0.1656008 ,
        0.        ]], dtype=float32)

In [36]:
testmybba = mybba(estimator=victim_policy)

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [37]:
testmybba.generate(np.expand_dims(obs, axis=0))

INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


array([[1.2575361e-01, 1.9123371e-01, 7.9478717e-01, 2.0521297e-01,
        9.9999982e-01, 5.5870229e-01, 6.0011965e-01, 5.3620988e-01,
        7.0538288e-01, 4.8258722e-01, 8.8098848e-01, 8.4765518e-01,
        5.8567822e-01, 7.3012257e-01, 1.1170563e-07, 8.3348423e-02,
        8.8911974e-01, 5.8766317e-02, 5.8766324e-02, 1.6369812e-01,
        9.1396046e-01, 5.8766276e-02, 4.1585621e-01, 3.3854747e-01,
        4.8285676e-07, 7.4196670e-08, 9.3136013e-01, 8.9069150e-02,
        3.0064498e-07, 8.9069128e-02, 2.1336032e-07]], dtype=float32)

In [38]:
%%timeit
testmybba.generate(np.expand_dims(obs, axis=0))

INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Using model predictions as correct labels for FGM.
INFO:KBMproject.mybb:Found initial adversarial image

3.17 s ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
bba.generate(np.expand_dims(obs, axis=0))

INFO:art.attacks.evasion.brendel_bethge:Using model predictions as correct labels for FGM.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for untargeted attack.
INFO:art.attacks.evasion.brendel_bethge:Using model predictions as correct labels for FGM.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for untargeted attack.
INFO:art.attacks.evasion.brendel_bethge:Using model predictions as correct labels for FGM.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for untargeted attack.
INFO:art.attacks.evasion.brendel_bethge:Using model predictions as correct labels for FGM.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for untargeted attack.
INFO:art.attacks.evasion.brendel_bethge:Using model predictions as correct labels for FGM.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for untargeted attack.
INFO:art.attacks.evasion.brendel_bethge:Using model predictions as cor

3.18 s ± 9.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


for successful targeted attack:

In [21]:
agent.predict(np.squeeze(obs), deterministic=True)

(array([17], dtype=int64), None)

In [28]:
kwargs = kwargs = dict(norm= np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        #batch_size=1,
        )

bbat = BrendelBethgeAttack(estimator=victim_policy, **kwargs)
testmybbat = mybba(estimator=victim_policy, **kwargs)

target = [10]

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.
INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [29]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.86 s ± 604 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
bbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.


The slowest run took 14.12 times longer than the fastest. This could mean that an intermediate result is being cached.
26.5 s ± 23 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
target = [9]

In [32]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.47 s ± 48.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
bbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.


INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.


3.35 s ± 36.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
target = [7]

In [35]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.46 s ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
bbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.


5.12 s ± 1.32 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
kwargs = dict(norm= np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=100_000, #default 100, finds sample matching the target class through iterative random search
        #batch_size=1,
        )

target = [12]

In [26]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.45 s ± 15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
bbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.
INFO:art.attacks.evasion.brendel_bethge:Found initial adversarial image for targeted attack.


6.09 s ± 3.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
agent.predict(np.squeeze(obs), deterministic=True)

(array([14], dtype=int64), None)

In [23]:
kwargs = kwargs = dict(norm= np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        batch_size=32,
        )

bbat = BrendelBethgeAttack(estimator=victim_policy, **kwargs)
testmybbat = mybba(estimator=victim_policy, **kwargs)

target = [10]

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.
INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [24]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.8 s ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
kwargs = kwargs = dict(norm= np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        batch_size=64,
        )

bbat = BrendelBethgeAttack(estimator=victim_policy, **kwargs)
testmybbat = mybba(estimator=victim_policy, **kwargs)

target = [10]

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.
INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [26]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.72 s ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
kwargs = kwargs = dict(norm= np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        batch_size=128,
        )

bbat = BrendelBethgeAttack(estimator=victim_policy, **kwargs)
testmybbat = mybba(estimator=victim_policy, **kwargs)

target = [10]

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.
INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [28]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.68 s ± 32.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
kwargs = kwargs = dict(norm= np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        batch_size=1000,
        )

bbat = BrendelBethgeAttack(estimator=victim_policy, **kwargs)
testmybbat = mybba(estimator=victim_policy, **kwargs)

target = [10]

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.
INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [30]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.65 s ± 7.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
kwargs = kwargs = dict(norm= np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        batch_size=10_000,
        )

bbat = BrendelBethgeAttack(estimator=victim_policy, **kwargs)
testmybbat = mybba(estimator=victim_policy, **kwargs)

target = [10]

INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.
INFO:art.estimators.classification.pytorch:Inferred 5 hidden layers on PyTorch classifier.


In [32]:
%%timeit
testmybbat.generate(np.expand_dims(obs, axis=0), 
y=to_categorical(target, nb_classes=agent.action_space[0].n))

INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.
INFO:KBMproject.mybb:Found initial adversarial image for targeted attack.


3.65 s ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Results above seem to indicate that increasing the batch size has a small effect on the computation time